In [5]:
pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import chardet

In [31]:
# Cargar el archivo train.csv desde la misma carpeta
E1 = pd.read_csv('2023_Accidentalidad.csv', sep=None, engine='python')
E2 = pd.read_csv('2023_LISTADO_UBICACIONES_NP.csv', sep=None, engine='python')
E3 = pd.read_csv('Actuaciones_limpieza_urbana_2023.csv', sep=None, engine='python')
E4 = pd.read_csv('Actuaciones_limpieza_urbana_2024.csv', sep=None, engine='python')
E5 = pd.read_csv('DATOS_ESTACIONES_JULIO_2024.csv', sep=None, engine='python')
E6 = pd.read_csv('DATOS_ESTACIONES_JUNIO_2024.csv', sep=None, engine='python')
E7 = pd.read_csv('intensidad_media_dias_laborables_2023-2024.csv', sep=None, engine='python')
E8 = pd.read_csv('Listado accidentes 2021.csv', sep=None, engine='python')
E9 = pd.read_csv('UbicacionEstacionesPermanentesSentidos.csv', sep=None, engine='python')
E10 = pd.read_csv('uso-de-autobuses.csv', sep=None, engine='python')

result = pd.concat([E1, E2, E3, E4, E5, E6, E7, E8, E9, E10], axis=0)

print(result.head())


  ﻿num_expediente       fecha     hora            localizacion numero  \
0     2023S000001  01/01/2023  1:15:00  AVDA. ALFONSO XIII, 33     33   
1     2023S000001  01/01/2023  1:15:00  AVDA. ALFONSO XIII, 33     33   
2     2023S000001  01/01/2023  1:15:00  AVDA. ALFONSO XIII, 33     33   
3     2023S000001  01/01/2023  1:15:00  AVDA. ALFONSO XIII, 33     33   
4     2023S000001  01/01/2023  1:15:00  AVDA. ALFONSO XIII, 33     33   

   cod_distrito   distrito tipo_accidente estado_meteorológico tipo_vehiculo  \
0           5.0  CHAMARTÍN        Alcance            Despejado  Todo terreno   
1           5.0  CHAMARTÍN        Alcance            Despejado  Todo terreno   
2           5.0  CHAMARTÍN        Alcance            Despejado  Todo terreno   
3           5.0  CHAMARTÍN        Alcance            Despejado  Todo terreno   
4           5.0  CHAMARTÍN        Alcance            Despejado  Todo terreno   

   ... Sentido Orient. ﻿Línea  Año Tipo de transporte  \
0  ...     NaN     NaN 

In [32]:
result.to_csv('resultado_concatenado.csv', index=False)


In [34]:
# Cargar el archivo train.csv desde la misma carpeta
td = pd.read_csv('resultado_concatenado.csv')

# Ver las primeras filas del archivo
td.head()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8440\4021125427.py:2: DtypeWarning: Columns (0,1,2,3,4,6,7,8,9,10,11,12,14,17,19,20,21,22,23,24,25,26,27,30,64,65,66,80,82,83,84,85,86,87,88,89,93,94,98,100,101,102,104,105,107) have mixed types. Specify dtype option on import or set low_memory=False.
  td = pd.read_csv('resultado_concatenado.csv')


,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,Sentido,Orient.,﻿Línea,Año,Tipo de transporte,Número anual de pasajeros,Expediciones por día laborable,Viajeros por día,Viajeros por expedición,Kilómetros anuales realizados
0,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
